<b> Computing Sharpe Ratio on existing shares allocation </b>

<pre>
Symbol  Shares  Acquired
AAPL    420     2014-07-07
GOOG    17      2014-06-26
TSLA    51      2015-03-03
TWTR    1396    2014-07-02
UNH     1464    2014-07-07
</pre>

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import datetime
from dateutil.parser import parse

import util

In [ ]:
#stocks_dir = '/Volumes/Photos/stocks/'
stocks_dir = './'

symbols = ['UNH', 'AAPL', 'GOOG', 'TSLA', 'TWTR']

stocks = {}  # key (str) - symbol, value - DataFrame

for s in symbols:
    file_name = stocks_dir + s + '.csv'
    df = pd.read_csv(file_name, index_col='Date', parse_dates=True, usecols=['Date', 'Adj. Close'])
    stocks[s] = df


sp500 = pd.read_csv('SP500_yahoo.csv', index_col='Date', parse_dates=True, na_values=['.'])
sp500 = sp500[['Adj Close']]
sp500.columns = ['Adj. Close']

symbols.extend(['SP500'])
stocks['SP500']  = sp500

stocks['TSLA'].head(3)

In [ ]:
shares = {'UNH': 1464, 'AAPL': 420, 'GOOG': 17, 'TSLA': 51, 'TWTR': 1396, 'SP500': 0}

for sym, df in stocks.items():
    df['shares'] = shares[sym]
    #df['daily_ret'] = df['Adj. Close'].pct_change(1)
    df['log_ret'] = np.log(df['Adj. Close']/df['Adj. Close'].shift(1))
    df['mv'] = df['shares'] * df['Adj. Close']
    df.drop(columns=['shares', 'Adj. Close'], inplace=True)
    df.columns = [ sym + ' ' + c for c in df.columns]

stocks['TSLA'].head(3)

In [ ]:
d1, d2 = '2014-04-01', '2018-03-15'
df = pd.concat(stocks.values(), axis=1).loc[d1:d2].copy()
df.head(3)

In [ ]:
df['mv'] = sum([df[sym + ' mv'] for sym in symbols])

for sym in symbols:
    df[sym + ' weight'] = df[sym + ' mv'] / df['mv']

df.head(3)

In [ ]:
#d1, d2 = '2014-07-02', '2018-01-01'
#d1, d2 = '2017-01-01', '2018-01-01'
#d1, d2 = '2015-03-01', '2015-06-01'
d1, d2 = '2015-01-01', '2017-01-01'


days = abs((parse(d1) - parse(d2)).days)
print("Real Return (x100%): {}".format( (df.loc[d1:d2].iloc[-1]['mv'] / df.loc[d1:d2].iloc[0]['mv']) ** ( 365/days ) - 1))

df2 = df.loc[d1:d2]

# Expected Return
log_ret = df2[[sym + ' log_ret' for sym in symbols]]
log_ret.columns = range(len(log_ret.columns))
#print(log_ret.head())

weights = df2[[sym + ' weight' for sym in symbols]]
weights.columns = range(len(weights.columns))
#print(weights.head())

exp_ret = (log_ret * weights).sum(axis=1).mean() * 252
print("Expected Portfolio Return (x100%): {}".format(exp_ret))

# Expected Volatility
exp_vol = np.sqrt((log_ret * weights).sum(axis=1).var() * 252)
print("Expected Volatility {}".format(exp_vol))
print('\n')
 
# Sharpe Ratio
SR = exp_ret/exp_vol
print('Sharpe Ratio {}'.format(SR))

In [ ]:
def sharpe(df, d1, d2):
    
    df = df.loc[d1:d2]
    
    # Expected Return
    log_ret = df[[sym + ' log_ret' for sym in symbols]]
    log_ret.columns = range(len(log_ret.columns))
    weights = df[[sym + ' weight' for sym in symbols]]
    weights.columns = range(len(weights.columns))
    exp_ret = (log_ret * weights).sum(axis=1).mean() * 252

    # Expected Volatility
    exp_vol = np.sqrt((log_ret * weights).sum(axis=1).var() * 252)

    # Sharpe Ratio
    SR = exp_ret/exp_vol
    return SR

sharpe(df, d1, d2)

In [ ]:
start_date, end_date, months = '2014-04-01', '2018-04-01', 12
it = util.generate_periods(start_date, end_date, months, months_overlap=0)

for _d1, _d2 in it:
    print(sharpe(df, _d1, _d2))

In [ ]:
from scipy.optimize import minimize

def get_ret_vol_sr(weights, df):
    """
    Takes in weights, returns array or return,volatility, sharpe ratio
    """
    weights = np.array(weights)
    log_ret = df[[sym + ' log_ret' for sym in symbols]]
    ret = (log_ret * weights).sum(axis=1).mean() * 252
    vol = np.sqrt((log_ret * weights).sum(axis=1).var() * 252)
    sr = ret/vol
    return np.array([ret,vol,sr])

def neg_sharpe(weights, df):
    return  get_ret_vol_sr(weights, df)[2] * -1

# Constraints
def check_sum(weights):
    '''
    Returns 0 if sum of weights is 1.0
    '''
    return np.sum(weights) - 1

# By convention of minimize function it should be a function that returns zero for conditions
cons = ({'type':'eq','fun': check_sum})
# 0-1 bounds for each weight
bounds = ((0,1),) * len(symbols)
# Initial Guess (equal distribution)
init_guess = [1/len(symbols)] * len(symbols)

In [ ]:
# Sequential Least SQuares Programming (SLSQP).
df2 = df.loc[d1:d2]
opt_results = minimize(neg_sharpe, init_guess, args=(df2), method='SLSQP', bounds=bounds, constraints=cons)

In [ ]:
opt_results

In [ ]:
print(symbols)
print(np.round(opt_results.x, 3))

In [ ]:
print("ret vol sr")
print(np.round(get_ret_vol_sr(opt_results.x, df2), 3))

In [ ]:
def generate_optimal_weights(df, date_from, date_to, freq=3, lookback=6):
    if (parse(date_from) - df.index[0]).days < 29 * lookback:
        raise Exception("Cannot lookback {} months from {}, because Time Series starts at {}".format(lookback, date_from, df.index[0]))
    if parse(date_to) >= df.index[-1]:
        raise Exception("date_to {} is after the end of Time Series {}".format(date_to, df.index[-1]))
    d1 = parse(date_from) - datetime.timedelta(30 * lookback)
    d2 = parse(date_from)
    
    i, max = 0, 10
    while d2 <= parse(date_to) and i < max:
        i += 1
        df2 = df.loc[d1:d2]
        print("Minimizing for dates {} {}". format(d1, d2))
        opt_results = minimize(neg_sharpe, init_guess, args=(df2), method='SLSQP', bounds=bounds, constraints=cons)

        weights = opt_results.x
        print("Symbols: {}".format(symbols))
        print("Weights: {}".format(np.round(weights, 5)))
        ret, vol, sr = get_ret_vol_sr(weights, df2)
        print("ret: {}, vol: {}, sr: {}".format(ret, vol, sr))
        
        # apply weights to df after_date
        df.loc[d2:, [s + ' weight' for s in symbols] ] = weights
        
        d1 = datetime.datetime.combine(util.add_months(d1, freq), datetime.datetime.min.time())
        d2 = datetime.datetime.combine(util.add_months(d2, freq), datetime.datetime.min.time())
        #print(opt_results)

In [ ]:
d1, d2 = '2015-04-01', '2018-01-01'
freq = 1
lookback = 3

df3 = df.loc[util.add_months(parse(d1), -lookback) : parse(d2) + datetime.timedelta(5)].copy()

In [ ]:
_df = df3.loc[d1:d2].copy()

# Expected Return
log_ret = _df[[sym + ' log_ret' for sym in symbols]]
log_ret.columns = range(len(log_ret.columns))
#print(log_ret.head())

weights = _df[[sym + ' weight' for sym in symbols]]
weights.columns = range(len(weights.columns))
#print(weights.head())

exp_ret = (log_ret * weights).sum(axis=1).mean() * 252
print("Expected Portfolio Return (x100%): {}".format(exp_ret))

# Expected Volatility
exp_vol = np.sqrt((log_ret * weights).sum(axis=1).var() * 252)
print("Expected Volatility {}".format(exp_vol))
print('\n')
 
# Sharpe Ratio
SR = exp_ret/exp_vol
print('Sharpe Ratio {}'.format(SR))

In [ ]:
generate_optimal_weights(df3, d1, d2, freq=freq, lookback=lookback)

In [ ]:
_df = df3.loc[d1:d2].copy()

# Expected Return
log_ret = _df[[sym + ' log_ret' for sym in symbols]]
log_ret.columns = range(len(log_ret.columns))
#print(log_ret.head())

weights = _df[[sym + ' weight' for sym in symbols]]
weights.columns = range(len(weights.columns))
#print(weights.head())

exp_ret = (log_ret * weights).sum(axis=1).mean() * 252
print("Expected Portfolio Return (x100%): {}".format(exp_ret))

# Expected Volatility
exp_vol = np.sqrt((log_ret * weights).sum(axis=1).var() * 252)
print("Expected Volatility {}".format(exp_vol))
print('\n')
 
# Sharpe Ratio
SR = exp_ret/exp_vol
print('Sharpe Ratio {}'.format(SR))

In [ ]:
df3.loc['2015-03-29':'2015-04-05'].head(20)